In [1]:
import dspy
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# llm = dspy.OpenAI(model='gpt-3.5-turbo-0125', api_key=OPENAI_API_KEY, max_tokens=300)
llm = dspy.OllamaLocal("zephyr-7b-beta", model_type="text", stop=['\n\n'], max_tokens=300)
dspy.settings.configure(lm=llm, temperature=0.7)

In [4]:
llm("hi how are you?")

["\ni'm doing well, thanks for asking. I hope you're doing okay too!"]

## Signatures

#### Sentiment Classification 

In [5]:
sentence = "it's a charming and often affecting journey."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment')
classify(sentence=sentence).sentiment

'positive'

In [6]:
llm.inspect_history(n=1)





Given the fields `sentence`, produce the fields `sentiment`.

---

Follow the following format.

Sentence: ${sentence}
Sentiment: ${sentiment}

---

Sentence: it's a charming and often affecting journey.
Sentiment: positive





In [7]:
short_story = dspy.Predict('sentence -> random_characters')
short_story(sentence=sentence).random_characters

'qxhjgkzl'

#### Summarization

In [8]:
# Example from the XSum dataset.
document = """The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."""

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)

print(response.summary)

The 21-year-old footballer, who played seven times for West Ham United last season and scored against FC Lustrains in a Europa League qualifier, had loan spells at Blackpool and Colchester United during the same campaign. He netted twice for Colchester but they were relegated. The youngster has now signed for a promoted team, but the length of his contract is unknown.


In [9]:
llm.inspect_history(n=1)





Given the fields `document`, produce the fields `summary`.

---

Follow the following format.

Document: ${document}
Reasoning: Let's think step by step in order to ${produce the summary}. We ...
Summary: ${summary}

---

Document: The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page.
Reasoning: Let's think step by step in order to summarize the career of a young football player who recently signed for a promoted team. First, we need to know his age and previous clubs. He played seven times for West Ham United last season and scored one goal against FC Lustra

In [10]:
data = "./data/dogs.txt"
with open(data, "r") as f:
    document = f.read()

In [11]:
summarize = dspy.ChainOfThought('document -> tldr')
response = summarize(document=document)

print(response.tldr)

Feline Financial Bank is an exclusive bank for cats founded by Sir Purrington, with cat-friendly architecture, efficient tellers offering savings accounts and investment portfolios, and a high-security vault guarded by feline warriors. It's a symbol of feline independence and sophistication.


#### Classification

In [12]:
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""
    
    sentence = dspy.InputField()
    sentiment = dspy.OutputField()

sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"  # from dair-ai/emotion

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='Fear'
)

In [13]:
class Weather(dspy.Signature):
    """Classify weather as rainy, sunny, or about to rain."""

    sentence = dspy.InputField()
    weather = dspy.OutputField()

sentence = "They both were both very hungry and wanted to be home. Large dark clouds moved towards them from the horizon."

classify = dspy.Predict(Weather)
classify(sentence=sentence)

Prediction(
    weather='about to rain'
)

#### A metric that evaluates faithfulness to citations 

In [14]:
class CheckCitationFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""

    context = dspy.InputField(desc="facts here are assumed to be true")
    text = dspy.InputField()
    faithfulness = dspy.OutputField(desc="True/False indicating if text is faithful to context")

context = "The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."

text = "Lee scored 3 goals for Colchester United."

faithfulness = dspy.ChainOfThought(CheckCitationFaithfulness)
faithfulness(context=context, text=text)

Prediction(
    rationale='check if text is faithful to context.\n1) We need to find out how many goals did Lee score for Colchester United.\n2) According to the context, Lee scored twice for Colchester United.\n3) So, Lee scored a total of 2 goals for Colchester United.\n4) The text says that Lee scored 3 goals for Colchester United.\n5) Therefore, the text is not faithful to the context.',
    faithfulness='False'
)

In [15]:
document = """AMENDMENT OF SOLICITATION/MODIFICATION OF CONTRACT
2. Amendment/Modification No.

3. Effective Date

0002

4. Requisition/Purchase Req No.

2024MAR15

6. Issued By

Code

SPRDL1

1. Contract ID Code

Page

1 Of 2

Firm Fixed Price

5. Project No. (If applicable)

SEE SCHEDULE

7. Administered By (If other than Item 6)

Code

DLA LAND WARREN
MARCILLE NORDSTOG
WARREN, MI 48397-5000

EMAIL: MARCILLE.NORDSTOG@DLA.MIL

8. Name And Address Of Contractor (No., Street, City, County, State and Zip Code)

X

9A. Amendment Of Solicitation No.
SPRDL1-24-Q-0018

9B. Dated (See Item 11)
2024JAN03

10A. Modification Of Contract/Order No.

10B. Dated (See Item 13)
Code

Facility Code
11. THIS ITEM ONLY APPLIES TO AMENDMENTS OF SOLICITATIONS

X The above numbered solicitation is amended as set forth in item 14. The hour and date specified for receipt of Offers
X is extended,
is not extended. 2024APR02
Offers must acknowledge receipt of this amendment prior to the hour and date specified in the solicitation or as amended by one of the following methods:
(a) By completing items 8 and 15, and returning ____________
2 signed copies of the amendments: (b) By acknowledging receipt of this amendment on each copy of the
offer submitted; or (c) By separate letter or telegram which includes a reference to the solicitation and amendment numbers. FAILURE OF YOUR
ACKNOWLEDGMENT TO BE RECEIVED AT THE PLACE DESIGNATED FOR THE RECEIPT OF OFFERS PRIOR TO THE HOUR AND DATE
SPECIFIED MAY RESULT IN REJECTION OF YOUR OFFER. If by virtue of this amendment you desire to change an offer already submitted, such change
may be made by telegram or letter, provided each telegram or letter makes reference to the solicitation and this amendment, and is received prior to the opening
hour and date specified.
12. Accounting And Appropriation Data (If required)

13. THIS ITEM ONLY APPLIES TO MODIFICATIONS OF CONTRACTS/ORDERS
It Modifies The Contract/Order No. As Described In Item 14.
A. This Change Order is Issued Pursuant To:
The Changes Set Forth In Item 14 Are Made In
The Contract/Order No. In Item 10A.
B. The Above Numbered Contract/Order Is Modified To Reflect The Administrative Changes (such as changes in paying office, appropriation data, etc.) Set
Forth In Item 14, Pursuant To The Authority of FAR 43.103(b).
C. This Supplemental Agreement Is Entered Into Pursuant To Authority Of:
D. Other (Specify type of modification and authority)
E. IMPORTANT: Contractor
is not,
is required to sign this document and return _______________ copies to the Issuing Office.
14. Description Of Amendment/Modification (Organized by UCF section headings, including solicitation/contract subject matter where feasible.)
SEE SECOND PAGE FOR DESCRIPTION

Except as provided herein, all terms and conditions of the document referenced in item 9A or 10A, as heretofore changed, remains unchanged and in full force and
effect.
15A. Name And Title Of Signer (Type or print)
16A. Name And Title Of Contracting Officer (Type or print)

15B. Contractor/Offeror

15C. Date Signed

16B. United States Of America
By

(Signature of person authorized to sign)
NSN 7540-01-152-8070
PREVIOUS EDITIONS UNUSABLE

30-105-02

16C. Date Signed

/SIGNED/

(Signature of Contracting Officer)
STANDARD FORM 30 (REV. 10-83)
Prescribed by GSA FAR (48 CFR) 53.243

CONTINUATION SHEET

Reference No. of Document Being Continued

Page

SPRDL1-24-Q-0018

PIIN/SIIN

MOD/AMD 0002

Name of Offeror or Contractor:
SUPPLEMENTAL INFORMATION

Buyer Name: MARCILLE NORDSTOG
Buyer Office Symbol/Telephone Number: ZGAC/(586)467-1219
Type of Contract 1: Firm Fixed Price
Kind of Contract: Other

*** End of Narrative A0000 ***
Amendment 0002 to solicitation SPRDL1-24-R-0018 is to extend the solicitation due date from 14MAR2024 to 02APR2024.
All other terms and conditions of the basic contract remain unchanged and in full force and effect.

*** END OF NARRATIVE A0003 ***

2

of 2"""

In [16]:
class CheckDocumentClass(dspy.Signature):
    """Verify the document is not an amendment to a procurement soliciation."""

    context = dspy.InputField(desc="facts here are assumed to be true")
    text = dspy.InputField()
    document_class = dspy.OutputField(desc="True/False indicatating if text is faithful to context")

context = document[:4000]

text = "This is not an amendment."

document_class = dspy.ChainOfThought(CheckDocumentClass)
document_class(context=context, text=text)

Prediction(
    rationale='understand why this statement is true. First, we need to define what an amendment is. According to the text provided, an amendment is a change or modification made to a contract or solicitation. Second, we need to identify whether this document falls under the category of an amendment. In order to determine that, we need to analyze the content of this document in comparison with what an amendment typically entails.\nIn this case, the statement "This is not an amendment" is true because this document does not contain any changes or modifications made to a contract or solicitation. The reason for that is because this document states that it is not an amendment. Therefore, since this document does not state that it contains any changes or modifications made to a contract or solicitation, then by default, we can conclude that this document falls under the category of something other than an amendment.\nIn summary, in order for something to be considered as an ame

************************

In [17]:
document[:4000]

'AMENDMENT OF SOLICITATION/MODIFICATION OF CONTRACT\n2. Amendment/Modification No.\n\n3. Effective Date\n\n0002\n\n4. Requisition/Purchase Req No.\n\n2024MAR15\n\n6. Issued By\n\nCode\n\nSPRDL1\n\n1. Contract ID Code\n\nPage\n\n1 Of 2\n\nFirm Fixed Price\n\n5. Project No. (If applicable)\n\nSEE SCHEDULE\n\n7. Administered By (If other than Item 6)\n\nCode\n\nDLA LAND WARREN\nMARCILLE NORDSTOG\nWARREN, MI 48397-5000\n\nEMAIL: MARCILLE.NORDSTOG@DLA.MIL\n\n8. Name And Address Of Contractor (No., Street, City, County, State and Zip Code)\n\nX\n\n9A. Amendment Of Solicitation No.\nSPRDL1-24-Q-0018\n\n9B. Dated (See Item 11)\n2024JAN03\n\n10A. Modification Of Contract/Order No.\n\n10B. Dated (See Item 13)\nCode\n\nFacility Code\n11. THIS ITEM ONLY APPLIES TO AMENDMENTS OF SOLICITATIONS\n\nX The above numbered solicitation is amended as set forth in item 14. The hour and date specified for receipt of Offers\nX is extended,\nis not extended. 2024APR02\nOffers must acknowledge receipt of this a

In [18]:
obliterate = dspy.Predict('sentence -> nonsense')
response = obliterate(sentence="Hi how are you, my name is Daniel Johnston")
print(response.nonsense)

Hj hoy ar yew, mz nm zjohnston


In [19]:
class SummarizeDocument(dspy.Signature):
    """Summarize the document for a potential contractor."""

    context = dspy.InputField(desc="requested information for summary")
    text = dspy.InputField()
    document_summary = dspy.OutputField(desc="Summary of the document with important details.")

context = document
text = "What type of payout does this have?"
document_summary = dspy.ChainOfThought(SummarizeDocument)
document_summary(context=context, text=text)



Prediction(
    rationale='understand the payout structure of this bond.',
    document_summary='This is a supplemental agreement for a modification of an existing contract between the United States government and a private company. The document outlines the changes being made to the original contract, including administrative provisions, technical provisions, and pricing information. It also includes details about the due date for the solicitation and the signatures of authorized individuals from both parties.'
)

In [20]:
llm.inspect_history(n=1)





Summarize the document for a potential contractor.

---

Follow the following format.

Context: requested information for summary

Text: ${text}

Reasoning: Let's think step by step in order to ${produce the document_summary}. We ...

Document Summary: Summary of the document with important details.

---

Context:
AMENDMENT OF SOLICITATION/MODIFICATION OF CONTRACT
2. Amendment/Modification No.

3. Effective Date

0002

4. Requisition/Purchase Req No.

2024MAR15

6. Issued By

Code

SPRDL1

1. Contract ID Code

Page

1 Of 2

Firm Fixed Price

5. Project No. (If applicable)

SEE SCHEDULE

7. Administered By (If other than Item 6)

Code

DLA LAND WARREN
MARCILLE NORDSTOG
WARREN, MI 48397-5000

EMAIL: MARCILLE.NORDSTOG@DLA.MIL

8. Name And Address Of Contractor (No., Street, City, County, State and Zip Code)

X

9A. Amendment Of Solicitation No.
SPRDL1-24-Q-0018

9B. Dated (See Item 11)
2024JAN03

10A. Modification Of Contract/Order No.

10B. Dated (See Item 13)
Code

Facility Code
11.

### Minimal Working Example

In [21]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

# Load math questions from the GSM8K dataset
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 37784.72it/s]


#### Define Module


In [22]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)

In [23]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset, valset=gsm8k_devset)

100%|██████████| 10/10 [00:40<00:00,  4.03s/it]

Bootstrapped 3 full traces after 10 examples in round 0.


In [24]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=gsm8k_devset, metric=gsm8k_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

Average Metric: 1 / 10  (10.0): 100%|██████████| 10/10 [00:38<00:00,  3.88s/it]

Average Metric: 1 / 10  (10.0%)



/home/peter-legion-wsl2/peter-projects/daybreak-ml-docs/.venv/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


10.0

In [25]:
llm.inspect_history(n=3)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: A third of the contestants at a singing competition are female, and the rest are male. If there are 18 contestants in total, how many of them are male?
Reasoning: Let's think step by step in order to find out how many males are there. We know that a third of the contestants are females, so we can calculate the number of females. Then, we subtract the number of females from the total number of contestants to get the number of males.
Answer: 12

---

Question: Nancy bought a pie sliced it into 8 pieces. She gave 1/2 to Joe and Darcy, and she gave 1/4 to Carl. How many slices were left?
Reasoning: Let's think step by step in order to find out how many slices were left. Firstly, we know that Nancy bought a pie and sliced it into 8 pieces. She gave Joe and Darcy 

***

In [26]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [27]:
pot = dspy.ProgramOfThought(GenerateAnswer)

In [28]:
question = "Sarah has 5 apples. She buys 7 more apples from the store. How many apples does Sarah have now?"
result = pot(question=question)

12


In [29]:
print(f"Question: {question}")
print(f"Final Predicted Answer (after ProgramOfThought process): {result.answer}")

Question: Sarah has 5 apples. She buys 7 more apples from the store. How many apples does Sarah have now?
Final Predicted Answer (after ProgramOfThought process): Sarah has 12 apples now.


In [ ]:
llm.inspect_history(n=3)

### Autonomous Coding

In [42]:
class CreateCode(dspy.Signature):
    "Print the python code to execute the requested code"

    question = dspy.InputField(desc="A description of the code to be generated")
    answer = dspy.OutputField(desc="The code to be executed")

question = "Add the numbers 9 and 13 and print the result"

answer = dspy.Predict(CreateCode)


In [43]:
answer(question=question)

Prediction(
    answer='```python\nx = 9\ny = 13\nresult = x + y\nprint(result)\n```'
)

In [44]:
print(answer(question=question).answer)

```python
x = 9
y = 13
result = x + y
print(result)
```


In [51]:
class CleanCode(dspy.Signature):
    "Correctly format the python code for exectution."

    question = dspy.InputField(desc="The python code to be formatted")
    answer = dspy.OutputField(desc="Cleaned python code, keeping newlines, but removing ``` or other non-pythonic items")


python_question = "Add the numbers 9 and 13 and print the result"

